# Simple Index Demo + ChatGPT

Use a very simple wrapper around the ChatGPT API

#### Load documents, build the GPTSimpleVectorIndex

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from gpt_index import GPTSimpleVectorIndex, SimpleDirectoryReader
from IPython.display import Markdown, display

In [2]:
# load documents
documents = SimpleDirectoryReader('../paul_graham_essay/data').load_data()

In [3]:
# use ChatGPT [beta]
from gpt_index.langchain_helpers.chatgpt import ChatGPTLLMPredictor

llm_predictor = ChatGPTLLMPredictor()

In [ ]:
index = GPTSimpleVectorIndex(documents, llm_predictor=llm_predictor)

#### Query Index

In [ ]:
# set Logging to DEBUG for more detailed outputs
response = index.query("What did the author do growing up?", llm_predictor=llm_predictor)

In [6]:
display(Markdown(f"<b>{response}</b>"))

<b>The new context doesn't provide information on what the author did growing up, so the original answer still stands as the best response to the question.</b>

In [7]:
response = index.query("What did the author do after his time at Y Combinator?", llm_predictor=llm_predictor)

INFO:root:> [query] Total LLM token usage: 3669 tokens
> [query] Total LLM token usage: 3669 tokens
INFO:root:> [query] Total embedding token usage: 14 tokens
> [query] Total embedding token usage: 14 tokens


In [8]:
display(Markdown(f"<b>{response}</b>"))

<b>After his time at Y Combinator, the author worked on painting and writing essays before returning to programming and writing a new Lisp called Bel. He also wrote a bunch of essays about topics he had stacked up and started to think about other things he could work on. In 2020, he wrote a personal essay about how he chooses what to work on, which turned out to be a surprising, messy answer. He decided to write a more detailed version for others to read, which became a final sentence in the essay.</b>